---

# ***Submission후처리 진행***

데이터에 noise 값이 너무 많이 존재한다고 판단해 시계열성을 전혀 띌 수 없는 데이터의 형태나 값을 고려해  데이터의 통계값으로 submission을 대체해주는 후처리를 진행하였습니다.  
  
Ex) 데이터의 값이 3주도 존재하지 않을때 / 실질적으로 데이터에 0이 너무많아 3주의 데이터를 가지고는 시계열성을 파악할 수 없음.

* [1. Import](#Import)
* [2. Read Data](#Read-Data)
* [3. 후처리](#후처리)

---
---

# ***Import***

In [454]:
import random
import os
import itertools
import pickle
from tqdm import tqdm
from collections import Counter

import pandas as pd
import numpy as np
import plotly.graph_objs as go
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from prophet import Prophet #prophet 라이브러리 불러오기

from prophet.diagnostics import cross_validation
from prophet.plot import plot_cross_validation_metric
from prophet.diagnostics import performance_metrics

---
---

# ***Read Data***

In [455]:
train_data = pd.read_csv(os.path.abspath("./data")+"/train.csv")
keyword = pd.read_csv(os.path.abspath("./data")+"/brand_keyword_cnt.csv")
product = pd.read_csv(os.path.abspath("./data")+"/product_info.csv")
sales = pd.read_csv(os.path.abspath("./data")+"/sales.csv")

In [456]:
sub_final = pd.read_csv(os.path.abspath("./submission")+"/deep_0.7_lgbm0.3.csv")

In [457]:
브랜드_lst = keyword['브랜드'].tolist()

---
---

# ***후처리***

* [ALL Zero Values](#ALL-Zero-Values)
* [Duplicate Product](#Duplicate-Product)
* [Only Two Values](#Only-Two-Values)
* [Zero Values In 2023](#Zero-Values-In-2023)
* [Only Two Values In 2023](#Only-Two-Values-In-2023)
* [Only Three Values](#Only-Three-Values)
* [Only Three Values In 2023](#Only-Three-Values-In-2023)
* [Only 7days Values](#Only-7days-Values)
* [Only 3Week Values](#Only-3Week-Values)
* [Only Last Day Value](#Only-Last-Day-Value)
* [Outlier Last 7Days](#Outlier-Last-7Days)


* [Except Non Missing values](#Except-Non-Missing-values)

***

In [458]:
len(train_data)

15890

---
---

### ***ALL Zero Values***
0으로 처리(85개)

In [459]:
all_zero_id_lst = train_data[train_data.iloc[:,6:].sum(axis=1)==0]['ID'].tolist()

In [460]:
len(all_zero_id_lst)

85

In [461]:
sub_final.loc[sub_final['ID'].isin(all_zero_id_lst), sub_final.columns[1:]] = 0

In [462]:
train_data = train_data[~train_data['ID'].isin(all_zero_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [463]:
15890 - 85, len(train_data)

(15805, 15805)

---
---

### ***Duplicate Product***
단종 제품 0으로 처리(40개)

In [464]:
train_data[train_data.duplicated('제품', keep=False)]['제품'].index

Int64Index([  113,   114,   214,   215,  3060,  3061,  4451,  4452,  4513,
             4514,  4810,  4811,  4846,  4847,  5319,  5320,  5767,  5768,
             5929,  5930,  5966,  5967,  5971,  5972,  5973,  5974,  5998,
             5999,  6853,  6854,  6856,  6857,  7131,  7132,  7133,  7134,
             7135,  7136,  7137,  7138,  7143,  7144,  7152,  7153,  7889,
             7890,  7939,  7940,  8333,  8334,  8654,  8655,  8658,  8659,
             9118,  9119,  9203,  9204,  9639,  9640,  9642,  9643, 10344,
            10345, 11698, 11699, 11993, 11994, 13162, 13163, 14300, 14301,
            14306, 14307, 14639, 14640, 14962, 14963, 15740, 15741],
           dtype='int64')

In [465]:
# 모든 id가 연속적임을 확인할 수 있음.
for i in range(0, len(train_data[train_data.duplicated('제품', keep=False)]['제품'].index), 2):
    idx = train_data[train_data.duplicated('제품', keep=False)]['제품'].index[i]
    idx_plus = train_data[train_data.duplicated('제품', keep=False)]['제품'].index[i+1]
    if train_data.loc[idx]['제품'] == train_data.loc[idx_plus]['제품']:
        pass
    else:
        print('중복되지 않습니다')

In [466]:
dup_id_lst = list(set(train_data[train_data.duplicated('제품', keep=False)]['ID'].tolist()))

dup_lst = list(set(train_data[train_data.duplicated('제품', keep=False)]['제품'].tolist()))

train_dup = train_data[train_data['제품'].isin(dup_lst)]
sales_dup = sales[sales['제품'].isin(dup_lst)]

In [467]:
len(dup_id_lst)

80

In [468]:
dup_0_id_lst = []
for i in range(0,len(dup_id_lst),2):
    dup_0_id_lst.append(train_dup.iloc[i,0])

In [469]:
dup_1_id_lst = []
for i in range(1,len(dup_id_lst),2):
    dup_1_id_lst.append(train_dup.iloc[i,0])

***dup_0 이 단종 되고 dup_1로 바뀐것으로 판단 가능.***

In [470]:
df_dup = pd.DataFrame()

for i in range(1,len(dup_id_lst),2):
    data_0 = pd.DataFrame(train_dup.set_index('ID').iloc[i,5:].T)
    # 0으로 출발값들 전부 제거
    while (data_0.iloc[0:, :] == 0).iloc[0,0]:
        data_0 = data_0.iloc[1:, :]
        
    data_0 = data_0.T
    data_0.reset_index(drop=True, inplace=True)

    data_1 = pd.DataFrame(train_dup.set_index('ID').iloc[i-1,5:].T)
    data_1 = data_1.T
    data_1.columns = pd.to_datetime(data_1.columns)
    data_1.reset_index(drop=True, inplace=True)
    
    start_date = pd.to_datetime(data_0.columns)[0].to_pydatetime()
    
    filtered_columns = data_1.loc[:, data_1.columns < start_date]
    filtered_columns.columns = filtered_columns.columns.strftime('%Y-%m-%d')
    
    tt = pd.concat([filtered_columns, data_0], axis = 1)
    df_dup = pd.concat([df_dup, tt], axis = 0)

In [471]:
df_dup.index = train_data[train_data['ID'].isin(dup_1_id_lst)].index

In [472]:
train_data.loc[train_data['ID'].isin(dup_1_id_lst), train_data.columns[6:]] = df_dup

In [473]:
train_data = train_data[~train_data['ID'].isin(dup_0_id_lst)]
train_data.reset_index(drop=True, inplace=True)

***단종된 제품들 0으로 처리***

In [474]:
sub_final.loc[sub_final['ID'].isin(dup_0_id_lst), sub_final.columns[1:]] = 0

In [475]:
15805 - 40, len(train_data)

(15765, 15765)

---
---

### ***Only Two Values***
0제외 값으로 처리(1287개)

In [476]:
subset = train_data.iloc[:, 6:]

# 각 행 별로 유일한 값의 개수를 세어서 결과를 Series로 저장
unique_counts = subset.nunique(axis=1)

# 유일한 값이 두 개밖에 없는 행 추출
rows_with_two_unique_values = subset[unique_counts == 2]

In [477]:
two_unique_lst = rows_with_two_unique_values.index

In [478]:
two_id_lst = train_data.loc[two_unique_lst]['ID'].tolist()

In [479]:
len(two_id_lst)

1287

In [480]:
# two_unique_lst에 해당하는 행 추출
selected_rows = train_data.loc[two_unique_lst].iloc[:,6:]

# 0을 제외한 각 행의 가장 많이 나온 값 찾기
most_common_values = selected_rows.apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [481]:
result_df = pd.DataFrame({'ID': two_id_lst, 'Most_Common_Value': most_common_values})

In [482]:
for i, row in result_df.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [483]:
all_true_rows = sub_final[sub_final['ID'].isin(two_id_lst)].iloc[:, 1:].apply(lambda row: row == row.iloc[0], axis=1).all(axis=1)
result = all_true_rows.all()
result

True

row 별로 전부 똑같은 값이 들어감을 확인 할 수 있음.

In [484]:
train_data = train_data[~train_data['ID'].isin(two_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [485]:
15765 - 1287, len(train_data)

(14478, 14478)

---
---

### ***Zero Values In 2023***
1로 처리(4631개)

In [486]:
zero_2023_id_lst = train_data[train_data.iloc[:,-94:].sum(axis=1)==0]['ID'].tolist()

In [487]:
len(zero_2023_id_lst)

4631

In [488]:
sub_final.loc[sub_final['ID'].isin(zero_2023_id_lst), sub_final.columns[1:]] = 1

In [489]:
train_data = train_data[~train_data['ID'].isin(zero_2023_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [490]:
14478 - 4631, len(train_data)

(9847, 9847)

---
---

### ***Only Two Values In 2023***
0 제외 최빈값으로 처리 (713개)

In [491]:
subset = train_data.iloc[:, -94:]

# 각 행 별로 유일한 값의 개수를 세어서 결과를 Series로 저장
unique_counts = subset.nunique(axis=1)

# 유일한 값이 두 개밖에 없는 행 추출
rows_with_two_unique_values_2023 = subset[unique_counts == 2]

In [492]:
two_unique_lst_2023 = rows_with_two_unique_values_2023.index

In [493]:
two_id_lst_2023 = train_data.loc[two_unique_lst_2023]['ID'].tolist()

In [494]:
len(two_id_lst_2023)

713

In [495]:
# two_unique_lst에 해당하는 행 추출
selected_rows_2023 = train_data.loc[two_unique_lst_2023].iloc[:, -94:]

# 0을 제외한 각 행의 가장 많이 나온 값 찾기
most_common_values_2023 = selected_rows_2023.apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [496]:
result_df_2023 = pd.DataFrame({'ID': two_id_lst_2023, 'Most_Common_Value': most_common_values_2023})

In [497]:
for i, row in result_df_2023.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [498]:
all_true_rows = sub_final[sub_final['ID'].isin(two_id_lst_2023)].iloc[:, 1:].apply(lambda row: row == row.iloc[0], axis=1).all(axis=1)
result = all_true_rows.all()
result

True

row 별로 전부 똑같은 값이 들어감을 확인 할 수 있음.

In [499]:
train_data = train_data[~train_data['ID'].isin(two_id_lst_2023)]
train_data.reset_index(drop=True, inplace=True)

In [500]:
9847 - 713, len(train_data)

(9134, 9134)

---
---

### ***Only Three Values***
(183개)

마지막 10일간 값이 없으면 그냥 0제외 두개중 최빈값으로 대체(94개)  
  
마지막 10일간 값이 존재하면 10일중 0제외 최빈값으로 대체(89개)



In [501]:
subset = train_data.iloc[:, 6:]

# 각 행 별로 유일한 값의 개수를 세어서 결과를 Series로 저장
unique_counts = subset.nunique(axis=1)

# 유일한 값이 세 개밖에 없는 행 추출
rows_with_three_unique_values = subset[unique_counts == 3]

In [502]:
three_unique_lst = rows_with_three_unique_values.index

In [503]:
three_id_lst = train_data.loc[three_unique_lst]['ID'].tolist()

In [504]:
len(three_id_lst)

183

*마지막 10일간 값이 없을때*     (94개)

In [505]:
f = rows_with_three_unique_values[rows_with_three_unique_values.iloc[:, -10:].eq(0).all(axis=1)]

In [506]:
f_lst = f.index.tolist()
f_id_lst = train_data.loc[f_lst]['ID'].tolist()

In [507]:
len(f_lst)

94

In [508]:
# two_unique_lst에 해당하는 행 추출
selected_rows_f = train_data.loc[f_lst].iloc[:,6:]

# 0을 제외한 각 행의 가장 많이 나온 값 찾기
most_common_values_f = selected_rows_f.apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [509]:
result_df_f = pd.DataFrame({'ID': f_id_lst, 'Most_Common_Value': most_common_values_f})

In [510]:
for i, row in result_df_f.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

최빈값이 중복일땐 마지막 값으로 대체

In [511]:
dup_idx_lst = []
for i in range(len(f)):
    f_T = f.iloc[i,:].T
    f_T = [value for value in f_T if value != 0]
    count_tt = 0
    counter_values = Counter(f_T)
    for value, count in counter_values.items():
        if count == counter_values.most_common(1)[0][1]:
            count_tt += 1
    if count_tt == 2:
        dup_idx_lst.append(f.iloc[i,:].name)

In [512]:
dup_id_lst = train_data.loc[dup_idx_lst]['ID'].tolist()

In [513]:
dup_value_lst = []

In [514]:
z = train_data[train_data['ID'].isin(dup_id_lst)]
for i in range(len(z)):
    z_T = pd.DataFrame(z.iloc[i, 6:])
    z_T = z_T[~(z_T == 0).all(axis=1)]
    dup_value_lst.append(z_T.iloc[-1,0])

In [515]:
result_df_dup = pd.DataFrame({'ID': dup_id_lst, 'Most_Common_Value': dup_value_lst})

In [516]:
for i, row in result_df_dup.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

*마지막 10일간 값이 있을때*  (89개)

In [517]:
f_10 = rows_with_three_unique_values[rows_with_three_unique_values.iloc[:,-10:].sum(axis=1)!=0]

In [518]:
f_lst_10 = f_10.index.tolist()
f_id_lst_10 = train_data.loc[f_lst_10]['ID'].tolist()

In [519]:
len(f_lst_10)

89

In [520]:
# two_unique_lst에 해당하는 행 추출
selected_rows_f_10 = train_data.loc[f_lst_10].iloc[:,6:]

# 0을 제외한 각 행의 가장 많이 나온 값 찾기
most_common_values_f_10 = selected_rows_f_10.apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [521]:
result_df_10 = pd.DataFrame({'ID': f_id_lst_10, 'Most_Common_Value': most_common_values_f_10})

In [522]:
for i, row in result_df_10.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

최빈값이 중복일땐 마지막 값으로 대체

In [523]:
dup_idx_lst_10 = []
for i in range(len(f_10)):
    f_T_10 = f_10.iloc[i,:].T
    f_T_10 = [value for value in f_T_10 if value != 0]
    count_tt_10 = 0
    counter_values_10 = Counter(f_T_10)
    for value, count in counter_values_10.items():
        if count == counter_values_10.most_common(1)[0][1]:
            count_tt_10 += 1
    if count_tt_10 == 2:
        dup_idx_lst_10.append(f_10.iloc[i,:].name)

In [524]:
dup_id_lst_10 = train_data.loc[dup_idx_lst_10]['ID'].tolist()

In [525]:
dup_value_lst_10 = []

In [526]:
z_10 = train_data[train_data['ID'].isin(dup_id_lst_10)]
for i in range(len(z_10)):
    z_T_10 = pd.DataFrame(z_10.iloc[i, 6:])
    z_T_10 = z_T_10[~(z_T_10 == 0).all(axis=1)]
    dup_value_lst_10.append(z_T_10.iloc[-1,0])

In [527]:
result_df_dup_10 = pd.DataFrame({'ID': dup_id_lst_10, 'Most_Common_Value': dup_value_lst_10})

In [528]:
for i, row in result_df_dup_10.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [529]:
train_data = train_data[~train_data['ID'].isin(f_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [530]:
train_data = train_data[~train_data['ID'].isin(f_id_lst_10)]
train_data.reset_index(drop=True, inplace=True)

In [531]:
9134 - 183, len(train_data)

(8951, 8951)

---
---

### ***Only Three Values In 2023***
(716개)  

마지막 7일간 값이 없으면 그냥 0제외 두개중 최빈값으로 대체(500개)

마지막 7일간 값이 존재하면 10일중 0제외 최빈값으로 대체(216개)

In [532]:
subset = train_data.iloc[:, -94:]

# 각 행 별로 유일한 값의 개수를 세어서 결과를 Series로 저장
unique_counts = subset.nunique(axis=1)

# 유일한 값이 세 개밖에 없는 행 추출
rows_with_three_unique_values = subset[unique_counts == 3]

In [533]:
three_unique_lst = rows_with_three_unique_values.index

In [534]:
three_id_lst = train_data.loc[three_unique_lst]['ID'].tolist()

In [535]:
len(three_id_lst)

716

*마지막 10일간 값이 없을때*     (94개)

In [536]:
f = rows_with_three_unique_values[rows_with_three_unique_values.iloc[:, -7:].eq(0).all(axis=1)]

In [537]:
f_lst = f.index.tolist()
f_id_lst = train_data.loc[f_lst]['ID'].tolist()

In [538]:
len(f_lst)

500

In [539]:
# two_unique_lst에 해당하는 행 추출
selected_rows_f = train_data.loc[f_lst].iloc[:,-94:]

# 0을 제외한 각 행의 가장 많이 나온 값 찾기
most_common_values_f = selected_rows_f.apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [540]:
result_df_f = pd.DataFrame({'ID': f_id_lst, 'Most_Common_Value': most_common_values_f})

In [541]:
for i, row in result_df_f.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

최빈값이 중복일땐 마지막 값으로 대체

In [542]:
dup_idx_lst = []
for i in range(len(f)):
    f_T = f.iloc[i,:].T
    f_T = [value for value in f_T if value != 0]
    count_tt = 0
    counter_values = Counter(f_T)
    for value, count in counter_values.items():
        if count == counter_values.most_common(1)[0][1]:
            count_tt += 1
    if count_tt == 2:
        dup_idx_lst.append(f.iloc[i,:].name)

In [543]:
dup_id_lst = train_data.loc[dup_idx_lst]['ID'].tolist()

In [544]:
dup_value_lst = []

In [545]:
z = train_data[train_data['ID'].isin(dup_id_lst)]
for i in range(len(z)):
    z_T = pd.DataFrame(z.iloc[i, 6:])
    z_T = z_T[~(z_T == 0).all(axis=1)]
    dup_value_lst.append(z_T.iloc[-1,0])

In [546]:
result_df_dup = pd.DataFrame({'ID': dup_id_lst, 'Most_Common_Value': dup_value_lst})

In [547]:
for i, row in result_df_dup.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

*마지막 7일간 값이 있을때*  (216개)

In [548]:
f_7 = rows_with_three_unique_values[rows_with_three_unique_values.iloc[:,-7:].sum(axis=1)!=0]

In [549]:
f_lst_7 = f_7.index.tolist()
f_id_lst_7 = train_data.loc[f_lst_7]['ID'].tolist()

In [550]:
len(f_lst_7)

216

In [551]:
# two_unique_lst에 해당하는 행 추출
selected_rows_f_7 = train_data.loc[f_lst_7].iloc[:,-7:]

# 0을 제외한 각 행의 가장 많이 나온 값 찾기
most_common_values_f_7 = selected_rows_f_7.apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [552]:
result_df_7 = pd.DataFrame({'ID': f_id_lst_7, 'Most_Common_Value': most_common_values_f_7})

In [553]:
for i, row in result_df_7.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

최빈값이 중복일땐 마지막 값으로 대체

In [554]:
dup_idx_lst_7 = []
for i in range(len(f_7)):
    f_T_7 = f_7.iloc[i,:].T
    f_T_7 = [value for value in f_T_7 if value != 0]
    count_tt_7 = 0
    counter_values_7 = Counter(f_T_7)
    for value, count in counter_values_7.items():
        if count == counter_values_7.most_common(1)[0][1]:
            count_tt_7 += 1
    if count_tt_7 == 2:
        dup_idx_lst_7.append(f_7.iloc[i,:].name)

In [555]:
dup_id_lst_7 = train_data.loc[dup_idx_lst_7]['ID'].tolist()

In [556]:
dup_value_lst_7 = []

In [557]:
z_7 = train_data[train_data['ID'].isin(dup_id_lst_7)]
for i in range(len(z_7)):
    z_T_7 = pd.DataFrame(z_7.iloc[i, 6:])
    z_T_7 = z_T_7[~(z_T_7 == 0).all(axis=1)]
    dup_value_lst_7.append(z_T_7.iloc[-1,0])

In [558]:
result_df_dup_7 = pd.DataFrame({'ID': dup_id_lst_7, 'Most_Common_Value': dup_value_lst_7})

In [559]:
for i, row in result_df_dup_7.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [560]:
train_data = train_data[~train_data['ID'].isin(three_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [561]:
8951 - 716, len(train_data)

(8235, 8235)

---
---

### ***Only 7days Values***
마지막 일주일에 값이 존재하지만 나머지에 값이 존재하지 않는 경우  
앞에서 0제외하고 똑같은 루틴 반복해 대입  
(17개)

In [562]:
only_7days = train_data[(train_data.iloc[:, 6:-7].sum(axis=1) == 0) & (train_data.iloc[:, -7:].sum(axis=1) != 0)]

In [563]:
only_7days_id_lst = only_7days['ID'].tolist()

In [564]:
len(only_7days_id_lst)

17

In [565]:
only_7days = only_7days.iloc[:, -7:]

In [566]:
forecast_list = []
for i in range(len(only_7days)):
    d = pd.DataFrame(only_7days.iloc[i])
    while (d.iloc[0,].values[0] == 0):
        d = d.iloc[1:, :]
    forecast_list.append(np.tile(d.values.squeeze(), 21))

In [567]:
forecast_df = pd.DataFrame(forecast_list).iloc[:,:21]

In [568]:
forecast_df.index = sub_final[sub_final['ID'].isin(only_7days_id_lst)].index
forecast_df.columns = sub_final[sub_final['ID'].isin(only_7days_id_lst)].iloc[:,1:].columns

In [569]:
sub_final.iloc[forecast_df.index,1:] = forecast_df.loc[forecast_df.index]

In [570]:
train_data = train_data[~train_data['ID'].isin(only_7days_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [571]:
8235 - 17, len(train_data)

(8218, 8218)

---
---

### ***Only 3Week Values***  
마지막 7일값 + 결측치 기준 앞 14일 값이 존재하지만 나머지에 값이 존재하지 않는 경우  
결측일 = 2023-02-23 ~ 2023-03-28

(178개)

마지막 7일 값이 0일때 ->> 앞에 존재하는 데이터의 패턴 대입  (21개)  
7일값 제외 앞의 값의 데이터 unique 3이하 값 ->> 마지막 7일 패턴 대입  (53개)  
마지막 7일 값 중 데이터 unique 가 0, 특정값 일때 ->> 특정값 대입  
((나머지 보류))


In [572]:
only_14days = train_data[(train_data.iloc[:, 6:-55].sum(axis=1) == 0) & (train_data.iloc[:, -55:].sum(axis=1) != 0)]

In [573]:
only_14days_id_lst = only_14days['ID'].tolist()

In [574]:
len(only_14days_id_lst)

178

***마지막 7일 값이 0*** (21개)

In [575]:
last_7_zero_id_lst = only_14days[only_14days.iloc[:,-7:].sum(axis=1)==0]['ID'].tolist()

In [576]:
len(last_7_zero_id_lst)

21

In [577]:
last_7 = only_14days[only_14days['ID'].isin(last_7_zero_id_lst)]
last_7 = last_7.iloc[:,6:]

In [578]:
forecast_list_last_7 = []
for i in range(len(last_7)):
    r = pd.DataFrame(last_7.iloc[i])
    while (r.iloc[0,].values[0] == 0):
        r = r.iloc[1:, :]
    while (r.iloc[-1,].values[0] == 0):
        r = r.iloc[:-1, :]
    forecast_list_last_7.append(np.tile(r.values.squeeze(), 21))

In [579]:
forecast_df_last_7 = pd.DataFrame(forecast_list_last_7).iloc[:,:21]

In [580]:
forecast_df_last_7.index = sub_final[sub_final['ID'].isin(last_7_zero_id_lst)].index
forecast_df_last_7.columns = sub_final[sub_final['ID'].isin(last_7_zero_id_lst)].iloc[:,1:].columns

In [581]:
sub_final.iloc[forecast_df_last_7.index,1:] = forecast_df_last_7.loc[forecast_df_last_7.index]

In [582]:
only_14days = only_14days[~only_14days['ID'].isin(last_7_zero_id_lst)]

In [583]:
len(only_14days)

157

***unique 3이하 처리*** (53개)

In [584]:
subset = only_14days.iloc[:, 6:-7]

# 각 행 별로 유일한 값의 개수를 세어서 결과를 Series로 저장
unique_counts = subset.nunique(axis=1)

# 유일한 값이 0제외 두 개밖에 없는 행 추출
rows_with_three_unique_values_only_14 = subset[unique_counts <= 3]

In [585]:
three_unique_lst_only_14 = rows_with_three_unique_values_only_14.index

three_id_lst_only_14 = train_data.loc[three_unique_lst_only_14]['ID'].tolist()

In [586]:
len(three_id_lst_only_14)

53

In [587]:
only_14days_unique3 = only_14days[only_14days['ID'].isin(three_id_lst_only_14)]
only_14days_unique3 = only_14days_unique3.iloc[:, -7:]

데이터가 특정값 두개면 0제외 최빈값 대입  
3개이하면 0 제거 패턴 대입  

In [588]:
unique_counts = only_14days_unique3.nunique(axis=1)
rows = only_14days_unique3[unique_counts == 2]

In [589]:
rows_idx = rows.index
rows_id = train_data.loc[rows_idx]['ID'].tolist()

In [590]:
len(rows_id)

2

In [591]:
select = only_14days_unique3.loc[rows_idx]

In [592]:
most_common = select.apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [593]:
result_df = pd.DataFrame({'ID': rows_id, 'Most_Common_Value': most_common})

In [594]:
for i, row in result_df.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [595]:
only_14days_unique3 = only_14days_unique3[~only_14days_unique3.index.isin(rows_idx)]

In [596]:
unique_counts = only_14days_unique3.nunique(axis=1)
rows = only_14days_unique3

In [597]:
rows_idx = rows.index
rows_id = train_data.loc[rows_idx]['ID'].tolist()

In [598]:
len(rows_id)

51

In [599]:
forecast_list = []
for i in range(len(rows)):
    r = pd.DataFrame(rows.iloc[i])
    r = r[~(r == 0).all(axis=1)]
    forecast_list.append(np.tile(r.values.squeeze(), 21))

In [600]:
forecast_df = pd.DataFrame(forecast_list).iloc[:,:21]

In [601]:
forecast_df.index = sub_final[sub_final['ID'].isin(rows_id)].index
forecast_df.columns = sub_final[sub_final['ID'].isin(rows_id)].iloc[:,1:].columns

In [602]:
sub_final.iloc[forecast_df.index,1:] = forecast_df.loc[forecast_df.index]

마지막 7일 값 0, 특정값 인 경우 (25개)

In [603]:
unique_counts = only_14days.iloc[:,-7:].nunique(axis=1)
rows = only_14days[unique_counts == 2]

In [604]:
rows_idx = rows.index
rows_id = train_data.loc[rows_idx]['ID'].tolist()

In [605]:
len(rows_id)

25

In [606]:
select = only_14days.loc[rows_idx]

In [607]:
most_common = select.iloc[:,-7:].apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [608]:
result_df = pd.DataFrame({'ID': rows_id, 'Most_Common_Value': most_common})

In [609]:
for i, row in result_df.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [610]:
only_14days = only_14days[~only_14days['ID'].isin(rows_id)]

마지막 7일 값 0, 특정값,특정값 인 경우

In [611]:
unique_counts = only_14days.iloc[:,-7:].nunique(axis=1)
rows = only_14days[unique_counts == 3]
rows = rows.iloc[:,-7:]

In [612]:
rows_idx = rows.index
rows_id = train_data.loc[rows_idx]['ID'].tolist()

In [613]:
len(rows_id)

43

In [614]:
forecast_list = []
for i in range(len(rows)):
    r = pd.DataFrame(rows.iloc[i])
    r = r[~(r == 0).all(axis=1)]
    forecast_list.append(np.tile(r.values.squeeze(), 21))

In [615]:
forecast_df = pd.DataFrame(forecast_list).iloc[:,:21]

In [616]:
forecast_df.index = sub_final[sub_final['ID'].isin(rows_id)].index
forecast_df.columns = sub_final[sub_final['ID'].isin(rows_id)].iloc[:,1:].columns

In [617]:
sub_final.iloc[forecast_df.index,1:] = forecast_df.loc[forecast_df.index]

In [618]:
only_14days = only_14days[~only_14days['ID'].isin(rows_id)]

In [619]:
len(only_14days)

89

나머지 89개에 대해서는 보간후 진행해야 할 것 같다.

In [620]:
train_data = train_data[~train_data['ID'].isin(only_14days_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [621]:
8218 - 178, len(train_data)

(8040, 8040)

---
---

### ***Only Last Day Value***  
결측치 기준 앞의 값이 0이면서 마지막 7일 값중 맨 마지막 값만 존재하는 경우  
결측일 = 2023-02-23 ~ 2023-03-28  
마지막 값 대체  

(18개)

In [622]:
last_value = train_data[(train_data.iloc[:,-45:-1].sum(axis=1) == 0) & (train_data.iloc[:, -7:].sum(axis=1) != 0)]

In [623]:
last_value_id_lst = last_value['ID'].tolist()

In [624]:
len(last_value_id_lst)

18

In [625]:
result_df_last_value = pd.DataFrame({'ID': last_value_id_lst, 'Value': last_value.iloc[:,-1]})

In [626]:
for i, row in result_df_last_value.iterrows():
    id_value = row['ID']
    value = row['Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [627]:
train_data = train_data[~train_data['ID'].isin(last_value_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [628]:
8040 - 18, len(train_data)

(8022, 8022)

---
---

### ***Outlier Last 7Days***  
##### 결측치 존재시.
마지막 7일의 평균이 2023년에서 마지막 7일을 제외한 값들의 평균의 3배 이상이면 시계열성을 띈다고 볼 수도 없고 이상치이다.  
마지막 7일 특정값 대체 or 패턴 대입

(322개)

In [629]:
train_non_7 = train_data[(train_data.iloc[:,-41:-7].sum(axis=1) == 0) & (train_data.iloc[:, -7:].sum(axis=1) != 0)]
selected_columns_last_7 = train_non_7.iloc[:, -7:]
selected_columns = train_non_7.iloc[:, 371:-7]

In [630]:
mean_lst = []
for i in range(len(selected_columns_last_7)):
    r = pd.DataFrame(selected_columns_last_7.iloc[i])
    r = r[~(r == 0).all(axis=1)]
    mean_lst.append(r.mean().values[0])
mean_df = pd.DataFrame(mean_lst)

In [631]:
mean_lst_not_7 = []
for i in range(len(selected_columns)):
    r = pd.DataFrame(selected_columns.iloc[i])
    r = r[~(r == 0).all(axis=1)]
    mean_lst_not_7.append(r.mean().values[0])
mean_df_not_7 = pd.DataFrame(mean_lst_not_7)

In [632]:
compare_mean = pd.concat([mean_df, mean_df_not_7], axis = 1)
compare_mean.columns = ['mean_7', 'mean_not_7']
compare_mean.index = train_non_7.index

In [633]:
compare_mean['v'] = None
compare_mean['v'] = compare_mean['mean_not_7']/compare_mean['mean_7']

In [634]:
outlier_7_id_lst = train_non_7.loc[compare_mean[compare_mean['v']>5].index]['ID'].tolist()

In [635]:
outlier_7 = train_data[train_data['ID'].isin(outlier_7_id_lst)]

In [636]:
len(outlier_7)

322

마지막 7일 값 0, 특정값 인 경우 (119개)

In [637]:
unique_counts = outlier_7.iloc[:,-7:].nunique(axis=1)
rows = outlier_7[unique_counts == 2]

In [638]:
rows_idx = rows.index
rows_id = train_data.loc[rows_idx]['ID'].tolist()

In [639]:
len(rows_id)

119

In [640]:
select = outlier_7.loc[rows_idx]

In [641]:
most_common = select.iloc[:,-7:].apply(lambda row: np.bincount(row[row != 0]).argmax(), axis=1)

In [642]:
result_df = pd.DataFrame({'ID': rows_id, 'Most_Common_Value': most_common})

In [643]:
for i, row in result_df.iterrows():
    id_value = row['ID']
    value = row['Most_Common_Value']
    sub_final.loc[sub_final['ID']==id_value, sub_final.columns != 'ID'] = value

In [644]:
outlier_7 = outlier_7[~outlier_7['ID'].isin(rows_id)]

마지막 7일 값중 5개가 0일때 (20개)

In [645]:
rows = outlier_7[(outlier_7.iloc[:, -7:] == 0).sum(axis=1) == 5]

In [646]:
rows_idx = rows.index
rows_id = train_data.loc[rows_idx]['ID'].tolist()

In [647]:
len(rows_id)

20

In [648]:
forecast_list = []
for i in range(len(rows)):
    r = pd.DataFrame(rows.iloc[i,-7:])
    while (r.iloc[0,].values[0] == 0):
        r = r.iloc[1:, :]
    while (r.iloc[-1,].values[0] == 0):
        r = r.iloc[:-1, :]
    forecast_list.append([r.iloc[-1,:].values[0]]*21)

In [649]:
forecast_df = pd.DataFrame(forecast_list).iloc[:,:21]

In [650]:
forecast_df.index = sub_final[sub_final['ID'].isin(rows_id)].index
forecast_df.columns = sub_final[sub_final['ID'].isin(rows_id)].iloc[:,1:].columns

In [651]:
sub_final.iloc[forecast_df.index,1:] = forecast_df.loc[forecast_df.index]

In [652]:
outlier_7 = outlier_7[~outlier_7['ID'].isin(rows_id)]

나머지 패턴 대입 (183개)

In [653]:
rows_idx = outlier_7.index
rows_id = train_data.loc[rows_idx]['ID'].tolist()

In [654]:
len(rows_id)

183

In [655]:
forecast_list = []
for i in range(len(outlier_7)):
    r = pd.DataFrame(outlier_7.iloc[i,-7:])
    r = r[~(r == 0).all(axis=1)]
    forecast_list.append(np.tile(r.values.squeeze(), 21))

In [656]:
forecast_df = pd.DataFrame(forecast_list).iloc[:,:21]

In [657]:
forecast_df.index = sub_final[sub_final['ID'].isin(rows_id)].index
forecast_df.columns = sub_final[sub_final['ID'].isin(rows_id)].iloc[:,1:].columns

In [658]:
sub_final.iloc[forecast_df.index,1:] = forecast_df.loc[forecast_df.index]

In [659]:
train_data = train_data[~train_data['ID'].isin(outlier_7_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [660]:
8022 - 322, len(train_data)

(7700, 7700)

---
---

### ***Except Non Missing values***  
결측일 = 2023-02-23 ~ 2023-03-28  
(1047개)

In [661]:
non_zero_id_lst = train_data[train_data.iloc[:,-41:-7].sum(axis=1)!=0]['ID'].tolist()

In [662]:
len(non_zero_id_lst)

1047

In [663]:
train_data = train_data[~train_data['ID'].isin(non_zero_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [664]:
7700 - 1047, len(train_data)

(6653, 6653)

In [665]:
train_raw = pd.read_csv(os.path.abspath("../data")+"/train.csv")

In [666]:
train_non_miss_value = train_raw[train_raw['ID'].isin(non_zero_id_lst)]
train_non_miss_value.reset_index(drop=True, inplace=True)

---
---

### ***Zero Values -43Days***  
(1614개)

In [667]:
zero_48_id_lst = train_data[train_data.iloc[:, -43:].sum(axis=1) == 0]['ID'].tolist()

In [668]:
zero_48 = train_data[train_data.iloc[:, -43:].sum(axis=1) == 0]

In [669]:
len(zero_48_id_lst)

1614

In [670]:
train_data = train_data[~train_data['ID'].isin(zero_48_id_lst)]
train_data.reset_index(drop=True, inplace=True)

In [671]:
len(train_data)

5039

In [672]:
6653 - 1614, len(train_data)

(5039, 5039)

***

In [673]:
sub_final

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15885,15885,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15886,15886,3,2,4,1,1,3,3,2,4,...,3,3,2,4,1,1,3,3,2,4
15887,15887,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
15888,15888,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [674]:
sub_prophet_3 = pd.read_csv(os.path.abspath("./submission")+"/대분류3번_역작_Prophet.csv")

In [675]:
train_raw = pd.read_csv(os.path.abspath("./data")+"/train.csv")

In [676]:
대분류3_id_lst = train_raw[train_raw['대분류'] == 'B002-C001-0003']['ID'].tolist()

In [677]:
대분류3_sub_idx = sub_final[sub_final['ID'].isin(대분류3_id_lst)].index

In [678]:
sub_final.loc[대분류3_sub_idx] = sub_prophet_3.loc[대분류3_sub_idx]

In [679]:
sub_final.to_csv('./submission/deep_0.7_lgbm0.3_plus_후처리_역작.csv', index = False)

***
***